In [55]:
print("DLModel")


# mlp for multi-output regression
from numpy import mean
from numpy import std
from numpy import asarray
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

max_models = 2
res_lst = []
model_lst = []


# get the dataset
def load_data(how_many=0):
	if how_many == 0:
		samples = 1000
	elif how_many == 1:
		samples = 100000
	elif how_many == 2:
		samples = 1000000

	X, y = make_regression(n_samples=samples, n_features=10, n_informative=5, n_targets=3, random_state=42)
	print("samples: ", samples)
	return X, y

# get the model
def get_model(n_inputs, n_outputs, num):
	if num == 0:
		lcl_model = Sequential()
		lcl_model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
		lcl_model.add(Dense(n_outputs))
		lcl_model.compile(loss='mae', optimizer='adam')
	elif num == 1:
		lcl_model = Sequential()
		lcl_model.add(Dense(22, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
		lcl_model.add(Dense(n_outputs))
		lcl_model.compile(loss='mae', optimizer='adam')
	return lcl_model



# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y, model_num):
	results = list()
	models = []

	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		n_inputs, n_outputs = X.shape[1], y.shape[1]
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]

		# define model
		model = get_model(n_inputs, n_outputs, model_num)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=5)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		# print('model_num: %d metrics_names: %s mae>%.3f' % (model_num, model.metrics_names, mae))
		results.append(mae)
		models.append(model)
	return mean(results), models[0]










DLModel


In [56]:
# For each test case
for test_case in range(3):
	# load dataset
	X, y = load_data(test_case)

	# Let's try each model
	best_result = 0.0
	best_model = None
	for curr_model in range(max_models):
		# evaluate model
		result, model = evaluate_model(X, y, curr_model)
		res_lst.append(result)
		model_lst.append(model)
		print('Model Number: %d metrics_names: %s Result: %.3f Model address: %s' % (curr_model, model.metrics_names, result, model))
		if result > best_result:
			best_result = result
			best_model = model


	print("best_result: %.3f best_model %s" % (best_result, best_model))

samples:  1000
Model Number: 0 metrics_names: ['loss'] Result: 66.434 Model address: <keras.engine.sequential.Sequential object at 0x00000200230FB9A0>
Model Number: 1 metrics_names: ['loss'] Result: 66.290 Model address: <keras.engine.sequential.Sequential object at 0x000002003270FD00>
best_result: 66.434 best_model <keras.engine.sequential.Sequential object at 0x00000200230FB9A0>
samples:  100000
Model Number: 0 metrics_names: ['loss'] Result: 1.316 Model address: <keras.engine.sequential.Sequential object at 0x000002002AD450A0>
Model Number: 1 metrics_names: ['loss'] Result: 1.147 Model address: <keras.engine.sequential.Sequential object at 0x000002001E9C8BE0>
best_result: 1.316 best_model <keras.engine.sequential.Sequential object at 0x000002002AD450A0>
samples:  1000000
Model Number: 0 metrics_names: ['loss'] Result: 0.026 Model address: <keras.engine.sequential.Sequential object at 0x000002001D7C0DC0>
Model Number: 1 metrics_names: ['loss'] Result: 0.025 Model address: <keras.engi

In [57]:
# make a prediction for new data
row = [-0.99859353,2.19284309,-0.42632569,-0.21043258,-1.13655612,-0.55671602,-0.63169045,-0.87625098,-0.99445578,-0.3677487]
newX = asarray([row])
yhat = best_model.predict(newX)
print('Predicted: %s' % yhat[0])

1/1 [==============================] - 0s 31ms/step
Predicted: [-253.91241 -190.95555 -158.85423]
